In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from stable_baselines3.common.logger import configure

from pathlib import Path

from flygym.envs.nmf_mujoco import MuJoCoParameters
from flygym.arena.mujoco_arena import FlatTerrain
from flygym.util.config import all_leg_dofs

from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

from visual_taxis import NMFVisualTaxis, MovingObjArena
from flygym.envs.nmf_mujoco import MuJoCoParameters

from util import linear_schedule, SaveIntermediateModelsCallback

c:\Users\Louise\Documents\EPFL\UPRAMDYA_intern\Flygym\flyvenv\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\Louise\Documents\EPFL\UPRAMDYA_intern\Flygym\flyvenv\lib\site-packages\torch\utils\tensorboard\__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [3]:
arena = MovingObjArena(obj_spawn_pos=(5, 3, 0), move_mode="s_shape")
# sim_params = MuJoCoParameters(render_playspeed=0.2, render_camera="birdseye_cam")
sim_params = MuJoCoParameters(render_playspeed=0.2)
sim = NMFVisualTaxis(
    sim_params=sim_params,
    arena=arena,
    decision_dt=0.05,
    n_stabilisation_steps=5000,
    obj_threshold=50,
)

In [4]:
log_dir = Path("logs")
log_dir.mkdir(parents=True, exist_ok=True)

callback = SaveIntermediateModelsCallback(check_freq=20_000, log_dir=log_dir)

mynmf = Monitor(sim, filename=str(log_dir / f"train_log_MLP"))

new_logger = configure(str(log_dir), ["stdout", "csv", "tensorboard"])

nmf_model = PPO(MlpPolicy, mynmf, verbose=True, learning_rate=linear_schedule(0.003))

nmf_model.set_logger(new_logger)

#print(nmf_model.policy)

nmf_model.learn(total_timesteps=400_000, progress_bar=False, callback=callback)
nmf_model.save(str(log_dir / f"saved_model_MLPlinearlr"))
mynmf.close()

Logging to logs
Using cpu device
Wrapping the env in a DummyVecEnv.
0
-1.6930208755353304
-1.1634716071746165
-1.661662633442977
-2.339824138272376
1.0661421005854361
1.5311430259372338
1.2066051114461844
0.1681913551277825
0.05591192321875604
1.1012703196338902


KeyboardInterrupt: 